In [21]:
# -*- coding:utf-8 -*-
import pandas as pd
import subprocess
import pydotplus
import matplotlib
import matplotlib.pyplot
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

# 数据导入
xls_file = pd.ExcelFile('alliance.xls')
# print(xls_file.sheet_names)
df = xls_file.parse('sheet1')
df_target = df['result']

In [22]:
# 对范畴类变量进行编码
for column in ['(L)HTTPstatus']:
    dummies = pd.get_dummies(df[column])
    df[dummies.columns]=dummies

# 去掉已编码的原属性
df.drop(['domain','(L)HTTPstatus'], axis=1, inplace=True)
#df.ix[:,df.columns!=999]

#去掉已编码的原属性
#df.drop(['domain','(L)HTTPstatus','(L)超链接数量','(L)外链数量','(L)有无外部js','(L)内部js匹配','(L)外部js匹配','(L)引用图片数量','(L)WebOCR','(R)超链接数量','(R)外链数量','(R)有无外部js','(R)内部js匹配','(R)外部js匹配','(R)引用图片数量','(R)WebOCR'], axis=1, inplace=True)
#df.ix[:,df.columns!=999]

In [23]:
# 拆分数据集为训练集和测试集
train, test = train_test_split(df, test_size = 0.2)   # doing split of training and testing
train_data, train_target= train.ix[:, train.columns != 'result'], train['result']
test_data, test_target = test.ix[:, test.columns != 'result'], test['result']
print(train_data.shape)
print(train_target.shape)

(1160, 20)
(1160,)


In [24]:
# GradientBoost
# 可控制参数为 n_estimators, max_depth
# For datasets with a large number of classes we strongly recommend to use RandomForestClassifier as an alternative to GradientBoostingClassifier .
#clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=2, random_state=0)
clf = GradientBoostingClassifier(loss='deviance', learning_rate=0.1, n_estimators=100, subsample=1.0, criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_split=1e-07, init=None, random_state=None, max_features=None, verbose=0, max_leaf_nodes=None, warm_start=False, presort='auto')
clf.fit(train_data,train_target)
clf.score(train_data, train_target)

0.97068965517241379

In [25]:
import pickle
pickle.dump(clf,open('pickled_model.p','wb'))

p = pickle.load(open('pickled_model.p','rb'))

In [26]:
# 交叉验证
# train_accuracy = cross_val_score(clf, train_data, train_target, cv=10,scoring='accuracy')
# print("Train_Accuracy >> \n",train_accuracy)
# train_recall = cross_val_score(clf, train_data, train_target, cv=10,scoring='recall')
# print("Train_Recall >> \n",train_recall)
# train_recall = cross_val_score(clf, train_data, train_target, cv=10,scoring='precision')
# print("Train_Precision >> \n",train_recall)
# train_recall = cross_val_score(clf, train_data, train_target, cv=10,scoring='f1')
# print("Train_F1 >> \n",train_recall)
# print("↑↑↑↑ Train ↑↑↑↑")
print("↓↓↓↓ Test ↓↓↓↓")
test_accuracy = cross_val_score(p, test_data, test_target, cv=10,scoring='accuracy')
print("Test_Accuracy >> \n",test_accuracy)
test_recall = cross_val_score(p, test_data, test_target, cv=10,scoring='recall')
print("Test_Recall >> \n",test_recall)
test_recall = cross_val_score(p, test_data, test_target, cv=10,scoring='precision')
print("Test_Precision >> \n",test_recall)
test_recall = cross_val_score(p, test_data, test_target, cv=10,scoring='f1')
print("Test_F1 >> \n",test_recall)

↓↓↓↓ Test ↓↓↓↓
Test_Accuracy >> 
 [ 0.93333333  0.86666667  0.86206897  0.96551724  0.93103448  0.96551724
  0.96551724  0.89655172  0.89655172  0.78571429]
Test_Recall >> 
 [ 0.90909091  0.81818182  0.63636364  0.90909091  0.81818182  1.          1.
  0.90909091  1.          0.7       ]
Test_Precision >> 
 [ 0.90909091  0.81818182  1.          1.          1.          0.91666667
  0.91666667  0.83333333  0.78571429  0.7       ]
Test_F1 >> 
 [ 0.90909091  0.81818182  0.77777778  0.95238095  0.9         0.95652174
  0.95652174  0.86956522  0.88        0.7       ]


In [27]:
# 数据导入
to_predict_file = pd.ExcelFile('predict.xls')
# print(xls_file.sheet_names)  # check what Sheet name the file has.
df = to_predict_file.parse('Sheet1')

# 对范畴类变量进行编码
for column in ['(L)HTTPstatus']:
    dummies = pd.get_dummies(df[column])
    df[dummies.columns]=dummies

# 去掉已编码的原属性
df.drop(['domain','(L)HTTPstatus','result'], axis=1, inplace=True)
#df.drop(['result','domain','(L)HTTPstatus','(L)超链接数量','(L)外链数量','(L)有无外部js','(L)内部js匹配','(L)外部js匹配','(L)引用图片数量','(L)WebOCR','(R)超链接数量','(R)外链数量','(R)有无外部js','(R)内部js匹配','(R)外部js匹配','(R)引用图片数量','(R)WebOCR'], axis=1, inplace=True)
predict_set = df
#进行预测
p.predict(predict_set)
#clf.predict_log_proba(predict_set)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 0,